# Imports

In [2]:
import pandas as pd
import numpy as np

# Init

In [3]:
data = pd.read_csv(r'cleaned_data.csv')
# data = pd.read_csv(r'') # For testing on my local machine

data.head()

,datetime,Timestamp,Open,High,Low,Close,Volume
0,2012-01-01 10:01:00,1.325412e+09,4.58,4.58,4.58,4.58,0.0
1,2012-01-01 10:02:00,1.325412e+09,4.58,4.58,4.58,4.58,0.0
2,2012-01-01 10:03:00,1.325412e+09,4.58,4.58,4.58,4.58,0.0
3,2012-01-01 10:04:00,1.325412e+09,4.58,4.58,4.58,4.58,0.0
4,2012-01-01 10:05:00,1.325412e+09,4.58,4.58,4.58,4.58,0.0


In [ ]:
# check perplexity